# 트리의 앙상블

## 랜덤포레스트

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

wine = pd.read_csv('https://bit.ly/wine-date')
wine.head()

,alcohol,sugar,pH,class
0,9.4,1.9,3.51,0.0
1,9.8,2.6,3.20,0.0
2,9.8,2.3,3.26,0.0
3,9.8,1.9,3.16,0.0
4,9.4,1.9,3.51,0.0


In [2]:
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

In [4]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier

# 무작위로 데이터를 샘플링(부트스트랩), 노드를 분할하기 위해 일부 feature를 무작위로 선택
# 여러 트리의 앙상블로 최종 결과를 예측
rf = RandomForestClassifier(n_jobs=-1, random_state=42)

# cross validation 결과
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9973541965122431 0.8903229806766861


In [5]:
# feature importance 확인 (과적합을 줄이고 일반화 능력을 키울 수 있음)
rf.fit(train_input, train_target)
print(rf.feature_importances_)

[0.23183515 0.50059756 0.26756729]


In [6]:
# OOB sample(부트스트랩에 포함되지 않은 샘플)을 활용해 성능 확인
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)
rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8945545507023283


## 엑스트라트리

In [7]:
from sklearn.ensemble import ExtraTreesClassifier

# 부트스트랩이 아닌 전체 데이터셋 사용, 노드는 무작위로 분할 (DecisionTree에서의 splitter='random'의 역할)
et = ExtraTreesClassifier(n_jobs=-1, random_state=42)

scores = cross_validate(et, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


In [8]:
et.fit(train_input, train_target)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


## 그레이디언트 부스팅

In [9]:
from sklearn.ensemble import GradientBoostingClassifier

# 작은 트리들의 앙상블로 예측 (이전 트리의 gradient를 기반으로 다음 트리를 업데이트)
gb = GradientBoostingClassifier(random_state=42)

scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


In [10]:
# 트리 개수를 늘려 학습
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2, random_state=42)

scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


In [11]:
gb.fit(train_input, train_target)
print(gb.feature_importances_)

[0.15881044 0.67988912 0.16130044]


## 히스토그램 기반 부스팅

In [12]:
from sklearn.ensemble import HistGradientBoostingClassifier

# feature를 256개의 구간으로 나누어 학습
hgb = HistGradientBoostingClassifier(random_state=42)

scores = cross_validate(hgb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9321723946453317 0.8801241948619236


In [13]:
hgb.fit(train_input, train_target)
print(rf.feature_importances_)

[0.23183515 0.50059756 0.26756729]


In [14]:
hgb.score(test_input, test_target)

0.8723076923076923

## 다양한 부스팅 알고리즘

#### XGBoost

In [18]:
# 설치가 필요한 경우 주석 해제
# !pip install xgboost
# !pip install lightgbm
# lightgbm 설치 오류 시 brew install libomp 로 라이브러리 설치 후 lightgbm 설치하기

In [19]:
from xgboost import XGBClassifier

xgb = XGBClassifier(tree_method='hist', random_state=42)

scores = cross_validate(xgb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9558403027491312 0.8782000074035686


#### LightGBM

In [20]:
from lightgbm import LGBMClassifier

lgb = LGBMClassifier(random_state=42)

scores = cross_validate(lgb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

[LightGBM] [Info] Number of positive: 3151, number of negative: 1006
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000349 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 370
[LightGBM] [Info] Number of data points in the train set: 4157, number of used features: 3
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.757999 -> initscore=1.141738
[LightGBM] [Info] Start training from score 1.141738
[LightGBM] [Info] Number of positive: 3151, number of negative: 1007
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000366 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 373
[LightGBM] [Info] Number of data points in the train set: 4158, number of used features: 3
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.757816 -> initscore=1.140744
[LightGBM] [Info] Start training from score 1.140744
[LightGBM] [Info] Number o